# Overview
Try to analyze and manipulate the data from _imports-85.data_ (CSV) and *train_data.json* to improve my data-munging skills in __*python*__ and __*Spark*__.

## Goal 1 - Find out how the risk breaks down by engine

Engine type is the fifth column and has values; `std` and `turbo`

In [1]:
rdd = spark.sparkContext.textFile('../data/imports-85.data').map(lambda line: line.split(','))
print rdd.count()
print rdd.take(2)

NameError: name 'spark' is not defined

In [ ]:
groupData = rdd.map(lambda line: ((line[0], line[4]), 1))
groupData.take(2)

In [ ]:
groupByRiskAndEngine = groupData.reduceByKey(lambda x, y: x+y)
groupByRiskAndEngine.cache()
groupByRiskAndEngine.take(2)

In [ ]:
print groupByRiskAndEngine.sortBy(lambda pair: int(pair[1]), False).collect()

In [ ]:
stdRiskAndEngine = groupByRiskAndEngine.filter(lambda pair: pair[0][1]=='std')
turboRiskAndEngine = groupByRiskAndEngine.filter(lambda pair: pair[0][1]=='turbo')

stdRiskAndEngine.cache()
turboRiskAndEngine.cache()

print('Sorted Risk for Standard Engines: \n%s\n' % (stdRiskAndEngine.sortBy(lambda pair: int(pair[1]), False).collect()))

print('Sorted Risk for Turbo Engines: \n%s' % (turboRiskAndEngine.sortBy(lambda pair: int(pair[1]), False).collect()))


In [ ]:
stdRisk = stdRiskAndEngine.map(lambda pair: int(pair[0][0]) * pair[1]).reduce(lambda x,y:x+y)
stdCount = stdRiskAndEngine.map(lambda pair: int(pair[1])).reduce(lambda x,y:x+y)
stdRiskWeighted = float(stdRisk)/float(stdCount)

turboRisk = turboRiskAndEngine.map(lambda pair: int(pair[0][0]) * pair[1]).reduce(lambda x,y:x+y)
turboCount = turboRiskAndEngine.map(lambda pair: int(pair[1])).reduce(lambda x,y:x+y)
turboRiskWeighted = float(turboRisk)/float(turboCount)

print('The std cummulative risk is: %d and the turbo cummulative risk is: %d' % (stdRisk, turboRisk))
print('The std weighted risk is (%d): %f and the turbo weighted risk is(%d): %f' % (stdCount, stdRiskWeighted, turboCount, turboRiskWeighted))

## Goal 2 - Evaluate Cross Val Model

In [1]:
import cPickle
import json
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from score_auto_gbm.FeatureTransformer import FeatureTransformer

In [2]:
with open("./score_auto_gbm/gbmFit.pkl", "rb") as pickle_file:
    gbmFit = cPickle.load(pickle_file)

In [3]:
data = pd.read_csv('../data/imports-85.data', header=None)
data.columns = [u'risk', u'aspiration', u'bodyStyle', u'bore', u'cityMPG', u'compressionRatio',
       u'curbWeight', u'driveWheels', u'engineLocation', u'engineSize',
       u'engineType', u'fuelSystem', u'fuelType', u'height', u'highwayMPG',
       u'horsepower', u'length', u'make', u'numCylinders', u'numDoors',
       u'peakRPM', u'price', u'risk', u'stroke', u'wheelBase', u'width']
data.head()

,risk,aspiration,bodyStyle,bore,cityMPG,compressionRatio,curbWeight,driveWheels,engineLocation,engineSize,...,length,make,numCylinders,numDoors,peakRPM,price,risk,stroke,wheelBase,width
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [4]:
trainData = data.ix[:,1:]

#use select to reorder 
trainData.head()

,aspiration,bodyStyle,bore,cityMPG,compressionRatio,curbWeight,driveWheels,engineLocation,engineSize,engineType,...,length,make,numCylinders,numDoors,peakRPM,price,risk,stroke,wheelBase,width
0,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [ ]:
type(trainData.ix[1,:])

In [8]:
for idx, row in enumerate(trainData.to_dict(orient='records')):
    print('Prediction: %s, Actual: %s.\n' % (list(gbmFit.predict(pd.DataFrame([row])))[0], data.ix[idx,0]))
    if idx > 5:
        break


ValueError: could not convert string to float: four

In [7]:
trainData.head().to_dict(orient='records')

[{u'aspiration': '?',
  u'bodyStyle': 'alfa-romero',
  u'bore': 'gas',
  u'cityMPG': 'std',
  u'compressionRatio': 'two',
  u'curbWeight': 'convertible',
  u'driveWheels': 'rwd',
  u'engineLocation': 'front',
  u'engineSize': 88.6,
  u'engineType': 168.8,
  u'fuelSystem': 64.1,
  u'fuelType': 48.8,
  u'height': 2548L,
  u'highwayMPG': 'dohc',
  u'horsepower': 'four',
  u'length': 130L,
  u'make': 'mpfi',
  u'numCylinders': '3.47',
  u'numDoors': '2.68',
  u'peakRPM': 9.0,
  u'price': '111',
  u'risk': '5000',
  u'stroke': 21L,
  u'wheelBase': 27L,
  u'width': '13495'},
 {u'aspiration': '?',
  u'bodyStyle': 'alfa-romero',
  u'bore': 'gas',
  u'cityMPG': 'std',
  u'compressionRatio': 'two',
  u'curbWeight': 'convertible',
  u'driveWheels': 'rwd',
  u'engineLocation': 'front',
  u'engineSize': 88.6,
  u'engineType': 168.8,
  u'fuelSystem': 64.1,
  u'fuelType': 48.8,
  u'height': 2548L,
  u'highwayMPG': 'dohc',
  u'horsepower': 'four',
  u'length': 130L,
  u'make': 'mpfi',
  u'numCylinders